---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

<h4>YC 答案</h4>

In [6]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open('university_towns.txt') as file:
        data = []
        for line in file:
            data.append(line[:-1])
            
    state_down = []
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        elif '(' in line:
            town = line[:(line.index('(')-1)]
            # 因为每次州开始时，那行没有 town 名
            # 如果把 state_down.append 写在最后
            # 则新的州，后面会添加之前的 town 名
            # 所以将 append 写在每次生成 town 之后
            state_down.append([state, town])
        else:
            town = line.strip()
            state_down.append([state, town]) 
    return pd.DataFrame(state_down, columns = ['State', 'RegionName'])

<h4>计算过程</h4>

In [3]:
with open('university_towns.txt') as file:
    data = []
    for line in file:
        data.append(line[:-1])
data[:2]

['Alabama[edit]', 'Auburn (Auburn University)[1]']

In [4]:
state_down = []
for line in data:
    if line[-6:] == '[edit]':
        state = line[:-6]
        #print("State :", state)
    elif '(' in line:
        town = line[:(line.index('(')-1)]
        #print("Town :", town)
        # 因为每次州开始时，那行没有 town 名
        # 如果把 state_down.append 写在最后
        # 则新的州，后面会添加之前的 town 名
        # 所以将 append 写在每次生成 town 之后
        state_down.append([state, town])
    else:
        town = line.strip()
        #print("Town 2 :", town)
        state_down.append([state, town]) 
state_down[:2]

[['Alabama', 'Auburn'], ['Alabama', 'Florence']]

In [5]:
df = pd.DataFrame(state_down, columns = ['State', 'RegionName'])
df.head(2)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence


<h4>YC 答案</h4>

In [11]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # only look at GDP data from the first quarter of 2000 onward.
    df = pd.read_excel('gdplev.xls', skiprows = 220, header = None)
    df = df.iloc[:,[4,5]]
    
    recession_start = []
    for i in range(len(df) - 4):
        if ((df.iloc[i][5] > df.iloc[i+1][5]) and (df.iloc[i+1][5] > df.iloc[i+2][5])):
            recession_start.append(df.iloc[i][4])
    
    return recession_start[0]

<h4>计算过程</h4>

In [8]:
# only look at GDP data from the first quarter of 2000 onward.
df = pd.read_excel('gdplev.xls', skiprows = 220, header = None)
df = df.iloc[:,[4,5]]
df.head()

,4,5
0,2000q1,10031.0
1,2000q2,10278.3
2,2000q3,10357.4
3,2000q4,10472.3
4,2001q1,10508.1


In [9]:
recession_start = []
for i in range(len(df) - 4):
    if ((df.iloc[i][5] > df.iloc[i+1][5]) and (df.iloc[i+1][5] > df.iloc[i+2][5])):
        recession_start.append(df.iloc[i][4])
recession_start

['2008q3', '2008q4']

In [10]:
recession_start[0]

'2008q3'

<h4>YC 答案</h4>

In [15]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    # only look at GDP data from the first quarter of 2000 onward.
    df = pd.read_excel('gdplev.xls', skiprows = 220, header = None)
    df = df.iloc[:,[4,5]]
    
    recession_end = []

    for i in range(len(df) - 4):
        if ((df.iloc[i][5] > df.iloc[i+1][5]) and (df.iloc[i+1][5] > df.iloc[i+2][5]) and 
           (df.iloc[i+2][5] < df.iloc[i+3][5]) and (df.iloc[i+3][5] < df.iloc[i+4][5])):
            recession_end.append([df.iloc[i][4], df.iloc[i+1][4], 
                                  df.iloc[i+2][4], df.iloc[i+3][4], df.iloc[i+4][4]])
            
    return recession_end[0][4]

<h4>计算过程</h4>

In [12]:
# only look at GDP data from the first quarter of 2000 onward.
df = pd.read_excel('gdplev.xls', skiprows = 220, header = None)
df = df.iloc[:,[4,5]]

In [13]:
recession_end = []

for i in range(len(df) - 4):
    if ((df.iloc[i][5] > df.iloc[i+1][5]) and (df.iloc[i+1][5] > df.iloc[i+2][5]) and 
       (df.iloc[i+2][5] < df.iloc[i+3][5]) and (df.iloc[i+3][5] < df.iloc[i+4][5])):
        recession_end.append([df.iloc[i][4], df.iloc[i+1][4], 
                               df.iloc[i+2][4], df.iloc[i+3][4], df.iloc[i+4][4]])
recession_end

[['2008q4', '2009q1', '2009q2', '2009q3', '2009q4']]

In [14]:
recession_end[0][4]

'2009q4'

<h4>YC 答案</h4>

In [17]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows = 220, header = None)
    df = df.iloc[:,[4,5]]
    
    recession_end = []

    for i in range(len(df) - 4):
        if ((df.iloc[i][5] > df.iloc[i+1][5]) and (df.iloc[i+1][5] > df.iloc[i+2][5]) and 
           (df.iloc[i+2][5] < df.iloc[i+3][5]) and (df.iloc[i+3][5] < df.iloc[i+4][5])):
            recession_end.append([df.iloc[i][4], df.iloc[i+1][4], 
                                  df.iloc[i+2][4], df.iloc[i+3][4], df.iloc[i+4][4]])
            
    return recession_end[0][2]

<h4>计算过程</h4>

In [16]:
# 因为出现 recession 后，连续 2 个 quarter GDP 回升，意味着 recession 结束。
# 所以倒数第 3 个 quarter 的 GDP 应该是最低的。
recession_end[0][2]

'2009q2'

<h4>YC 答案</h4>

In [25]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # 读取数据
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    # 仅提取 2000 年后，以及 State 和 RegionName 列
    list_col = [1, 2] + list(range(51,251))
    df = df.iloc[:, list_col]
    # 生成新 DataFrame, 先仅包含 State 和 RegionName 列
    # 直接用 df[['RegionName', 'State']] 会生成 SettingWithCopyWarning：
    # A value is trying to be set on a copy of a slice from a DataFrame.
    # Try using .loc[row_indexer,col_indexer] = value instead
    df_qrt = df.loc[:, ['RegionName', 'State']]
    # 因为 2016 不足最后两个季度，所以单列
    for year in range(2000,2016):
        df_qrt[str(year) + 'q1'] = df[[str(year) + '-01', str(year) + '-02', str(year) + '-03']].mean(axis = 1)
        df_qrt[str(year) + 'q2'] = df[[str(year) + '-04', str(year) + '-05', str(year) + '-06']].mean(axis = 1)
        df_qrt[str(year) + 'q3'] = df[[str(year) + '-07', str(year) + '-08', str(year) + '-09']].mean(axis = 1)
        df_qrt[str(year) + 'q4'] = df[[str(year) + '-10', str(year) + '-11', str(year) + '-12']].mean(axis = 1)
    # 2016 只有 8 个月，不能完整构成第 3，4 季度
    year = 2016
    df_qrt[str(year) + 'q1'] = df[[str(year) + '-01', str(year) + '-02', str(year) + '-03']].mean(axis = 1)
    df_qrt[str(year) + 'q2'] = df[[str(year) + '-04', str(year) + '-05', str(year) + '-06']].mean(axis = 1)
    df_qrt[str(year) + 'q3'] = df[[str(year) + '-07', str(year) + '-08']].mean(axis = 1)
    # Use this dictionary to map state names to two letter acronyms
    df_qrt['State'] = [states[State] for State in df_qrt['State']]
    # have a multi-index in the shape of ["State","RegionName"]
    df_qrt.set_index(['State', 'RegionName'], inplace = True)
    
    return df_qrt

<h4>计算过程</h4>

In [18]:
# 读取数据
df = pd.read_csv('City_Zhvi_AllHomes.csv')
df.head(2)

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100


In [19]:
# 仅提取 2000 年后，以及 State 和 RegionName 列
list_col = [1, 2] + list(range(51,251))
df = df.iloc[:, list_col]
df.head(2)

,RegionName,State,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,Los Angeles,CA,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100


In [20]:
# 生成新 DataFrame, 先仅包含 State 和 RegionName 列
# 直接用 df[['RegionName', 'State']] 会生成 SettingWithCopyWarning：
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead
df_qrt = df.loc[:, ['RegionName', 'State']]
df_qrt.head(2)

,RegionName,State
0,New York,NY
1,Los Angeles,CA


In [21]:
# 因为 2016 不足最后两个季度，所以单列
for year in range(2000,2016):
    df_qrt[str(year) + 'q1'] = df[[str(year) + '-01', str(year) + '-02', str(year) + '-03']].mean(axis = 1)
    df_qrt[str(year) + 'q2'] = df[[str(year) + '-04', str(year) + '-05', str(year) + '-06']].mean(axis = 1)
    df_qrt[str(year) + 'q3'] = df[[str(year) + '-07', str(year) + '-08', str(year) + '-09']].mean(axis = 1)
    df_qrt[str(year) + 'q4'] = df[[str(year) + '-10', str(year) + '-11', str(year) + '-12']].mean(axis = 1)
df_qrt.head(2)

,RegionName,State,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,...,2013q3,2013q4,2014q1,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,481366.666667,492433.333333,503133.333333,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333
1,Los Angeles,CA,207066.666667,214466.666667,220966.666667,226166.666667,233000.0,239100.0,245066.666667,253033.333333,...,455066.666667,471933.333333,484266.666667,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333


In [22]:
# 2016 只有 8 个月，不能完整构成第 3，4 季度
year = 2016
df_qrt[str(year) + 'q1'] = df[[str(year) + '-01', str(year) + '-02', str(year) + '-03']].mean(axis = 1)
df_qrt[str(year) + 'q2'] = df[[str(year) + '-04', str(year) + '-05', str(year) + '-06']].mean(axis = 1)
df_qrt[str(year) + 'q3'] = df[[str(year) + '-07', str(year) + '-08']].mean(axis = 1)
df_qrt.head(2)

,RegionName,State,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
1,Los Angeles,CA,207066.666667,214466.666667,220966.666667,226166.666667,233000.0,239100.0,245066.666667,253033.333333,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0


In [23]:
# Use this dictionary to map state names to two letter acronyms
df_qrt['State'] = [states[State] for State in df_qrt['State']]
df_qrt.head()

,RegionName,State,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
0,New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
1,Los Angeles,California,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
2,Chicago,Illinois,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
3,Philadelphia,Pennsylvania,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
4,Phoenix,Arizona,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [24]:
df_qrt.set_index(['State', 'RegionName'], inplace = True)
df_qrt.head(2)

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.0,239100.0,245066.666667,253033.333333,261966.666667,272700.0,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0


<h4>YC 答案</h4>

In [27]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"

<h4>计算过程</h4>